# BERT Tweet sentiment Classification


In [ ]:
# Load the Drive helper and mount
from google.colab import drive

# This will prompt for authorization.
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
#download bert pretrained model
# !wget "https://storage.googleapis.com/bert_models/2018_10_18/uncased_L-12_H-768_A-12.zip"

In [ ]:
# !unzip uncased_L-12_H-768_A-12.zip

In [ ]:
ls

drive/  sample_data/


In [ ]:
import os
import collections
import pandas as pd
import tensorflow as tf
import tensorflow_hub as hub
from datetime import datetime

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
## read data from google drive
DATASET_PATH = '/content/drive/MyDrive/training_csv'

## get a sample to acelerate training 
training_data = pd.read_csv(DATASET_PATH)


In [ ]:
training_data = training_data.sample(10000)

In [ ]:
training_data

,id,text,emotion
120274,0x249a43,Feeling <LH> this morning 😇,joy
587345,0x2ae90d,<LH> for all of the Advance evangelists groups...,anticipation
1108654,0x35b364,The world needs your #love. Nothing changes un...,joy
41878,0x2bb6b3,@DrVoyeur all <LH> bitch-asses ya uploading..N...,disgust
1358378,0x241f3e,@bairdjulia @ABCthedrum <LH> you @bairdjulia -...,joy
...,...,...,...
229557,0x26fbbe,God... You are Selfless! Thank you! <LH>,joy
1143938,0x38bccc,Been thinking about the new guy in my life all...,anticipation
534982,0x3271dd,MR ACOSTA! TRUMP AND SENSE OF DUTY IS AN OXYMO...,disgust
539894,0x2edd20,Just watched one guy try to hi-five and anothe...,sadness


In [ ]:
# possible_labels = training_data.emotion.unique()
# possible_labels
possible_labels = ['fear', 'joy', 'anticipation', 'trust', 'sadness', 'disgust',
		       'surprise', 'anger']
possible_labels
# array(['sadness', 'disgust', 'fear', 'joy', 'anticipation', 'trust',
#        'surprise', 'anger'], dtype=object)

['fear',
 'joy',
 'anticipation',
 'trust',
 'sadness',
 'disgust',
 'surprise',
 'anger']

In [ ]:

label_dict = {}
for index, possible_label in enumerate(possible_labels):
    label_dict[possible_label] = index

In [ ]:
label_dict

{'anger': 7,
 'anticipation': 2,
 'disgust': 5,
 'fear': 0,
 'joy': 1,
 'sadness': 4,
 'surprise': 6,
 'trust': 3}

In [ ]:
training_data['label'] = training_data.emotion.replace(label_dict)

In [ ]:
training_data

,id,text,emotion,label
120274,0x249a43,Feeling <LH> this morning 😇,joy,1
587345,0x2ae90d,<LH> for all of the Advance evangelists groups...,anticipation,2
1108654,0x35b364,The world needs your #love. Nothing changes un...,joy,1
41878,0x2bb6b3,@DrVoyeur all <LH> bitch-asses ya uploading..N...,disgust,5
1358378,0x241f3e,@bairdjulia @ABCthedrum <LH> you @bairdjulia -...,joy,1
...,...,...,...,...
229557,0x26fbbe,God... You are Selfless! Thank you! <LH>,joy,1
1143938,0x38bccc,Been thinking about the new guy in my life all...,anticipation,2
534982,0x3271dd,MR ACOSTA! TRUMP AND SENSE OF DUTY IS AN OXYMO...,disgust,5
539894,0x2edd20,Just watched one guy try to hi-five and anothe...,sadness,4


In [ ]:

from sklearn.model_selection import train_test_split

#train test split
X_train, X_val, y_train, y_val = train_test_split(training_data.index.values, 
                                                   training_data.label.values,
                                                   test_size = 0.1,
                                                   random_state = 17,
                                                   stratify = training_data.label.values)

In [ ]:
training_data['data_type'] = ['not_set'] * training_data.shape[0]

In [ ]:
#fill in data type
training_data.loc[X_train, 'data_type'] = 'train'
training_data.loc[X_val, 'data_type'] = 'val'

#groupby count
training_data.groupby(['emotion', 'label', 'data_type']).count()

id  text
emotion      label data_type            
anger        7     train       263   263
                   val          29    29
anticipation 2     train      1572  1572
                   val         175   175
disgust      5     train       845   845
                   val          94    94
fear         0     train       434   434
                   val          48    48
joy          1     train      3165  3165
                   val         352   352
sadness      4     train      1180  1180
                   val         131   131
surprise     6     train       331   331
                   val          37    37
trust        3     train      1210  1210
                   val         134   134

In [ ]:
training_data.set_index("id",inplace=True)
training_data

,text,emotion,label,data_type
id,,,,
0x249a43,Feeling <LH> this morning 😇,joy,1,train
0x2ae90d,<LH> for all of the Advance evangelists groups...,anticipation,2,train
0x35b364,The world needs your #love. Nothing changes un...,joy,1,train
0x2bb6b3,@DrVoyeur all <LH> bitch-asses ya uploading..N...,disgust,5,train
0x241f3e,@bairdjulia @ABCthedrum <LH> you @bairdjulia -...,joy,1,train
...,...,...,...,...
0x26fbbe,God... You are Selfless! Thank you! <LH>,joy,1,train
0x38bccc,Been thinking about the new guy in my life all...,anticipation,2,train
0x3271dd,MR ACOSTA! TRUMP AND SENSE OF DUTY IS AN OXYMO...,disgust,5,train


In [ ]:
training_data

,text,emotion,label,data_type
id,,,,
0x249a43,Feeling <LH> this morning 😇,joy,1,train
0x2ae90d,<LH> for all of the Advance evangelists groups...,anticipation,2,train
0x35b364,The world needs your #love. Nothing changes un...,joy,1,train
0x2bb6b3,@DrVoyeur all <LH> bitch-asses ya uploading..N...,disgust,5,train
0x241f3e,@bairdjulia @ABCthedrum <LH> you @bairdjulia -...,joy,1,train
...,...,...,...,...
0x26fbbe,God... You are Selfless! Thank you! <LH>,joy,1,train
0x38bccc,Been thinking about the new guy in my life all...,anticipation,2,train
0x3271dd,MR ACOSTA! TRUMP AND SENSE OF DUTY IS AN OXYMO...,disgust,5,train


In [ ]:
!pip install transformers

In [ ]:
from transformers import BertTokenizer
from torch.utils.data import TensorDataset
tokenizer = BertTokenizer.from_pretrained('bert-base-uncased',
                                         do_lower_case = True)

In [ ]:
#encode train set
encoded_data_train = tokenizer.batch_encode_plus(training_data[training_data.data_type == 'train'].text.values,
                                                add_special_tokens = True,
                                                return_attention_mask = True,
                                                pad_to_max_length = True,
                                                max_length = 150,
                                                return_tensors = 'pt')
                                                
#encode validation set
encoded_data_val = tokenizer.batch_encode_plus(training_data[training_data.data_type == 'val'].text.values,
                                                add_special_tokens = True,
                                                return_attention_mask = True,
                                                pad_to_max_length = True,
                                                max_length = 150,
                                                return_tensors = 'pt')

Truncation was not explicitly activated but `max_length` is provided a specific value, please use `truncation=True` to explicitly truncate examples to max length. Defaulting to 'longest_first' truncation strategy. If you encode pairs of sequences (GLUE-style) with the tokenizer you can select this strategy more precisely by providing a specific strategy to `truncation`.
/usr/local/lib/python3.7/dist-packages/transformers/tokenization_utils_base.py:2232: FutureWarning: The `pad_to_max_length` argument is deprecated and will be removed in a future version, use `padding=True` or `padding='longest'` to pad to the longest sequence in the batch, or use `padding='max_length'` to pad to a max length. In this case, you can give a specific length with `max_length` (e.g. `max_length=45`) or leave max_length to None to pad to the maximal input size of the model (e.g. 512 for Bert).
  FutureWarning,


In [ ]:
# !pip install torch

In [ ]:
import torch
#train set
input_ids_train = encoded_data_train['input_ids']
attention_masks_train = encoded_data_train['attention_mask']
labels_train = torch.tensor(training_data[training_data.data_type == 'train'].label.values)

#validation set
input_ids_val = encoded_data_val['input_ids']
attention_masks_val = encoded_data_val['attention_mask']
labels_val = torch.tensor(training_data[training_data.data_type == 'val'].label.values)

In [ ]:
input_ids_train

tensor([[ 101, 3110, 1026,  ...,    0,    0,    0],
        [ 101, 1026, 1048,  ...,    0,    0,    0],
        [ 101, 1996, 2088,  ...,    0,    0,    0],
        ...,
        [ 101, 2720, 9353,  ...,    0,    0,    0],
        [ 101, 2074, 3427,  ...,    0,    0,    0],
        [ 101, 3492, 2469,  ...,    0,    0,    0]])

In [ ]:
attention_masks_train

tensor([[1, 1, 1,  ..., 0, 0, 0],
        [1, 1, 1,  ..., 0, 0, 0],
        [1, 1, 1,  ..., 0, 0, 0],
        ...,
        [1, 1, 1,  ..., 0, 0, 0],
        [1, 1, 1,  ..., 0, 0, 0],
        [1, 1, 1,  ..., 0, 0, 0]])

In [ ]:
labels_train

tensor([1, 2, 1,  ..., 5, 4, 1])

In [ ]:

from transformers import BertForSequenceClassification

model = BertForSequenceClassification.from_pretrained('bert-base-uncased',
                                                      num_labels = len(label_dict),
                                                      output_attentions = False,
                                                      output_hidden_states = False)

Some weights of the model checkpoint at bert-base-uncased were not used when initializing BertForSequenceClassification: ['cls.seq_relationship.bias', 'cls.predictions.transform.dense.bias', 'cls.predictions.bias', 'cls.predictions.transform.LayerNorm.weight', 'cls.predictions.transform.LayerNorm.bias', 'cls.predictions.transform.dense.weight', 'cls.predictions.decoder.weight', 'cls.seq_relationship.weight']
- This IS expected if you are initializing BertForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of BertForSequenceClassification were not initialized from the model checkpoint at

In [ ]:
#model summary
model.config

BertConfig {
  "_name_or_path": "bert-base-uncased",
  "architectures": [
    "BertForMaskedLM"
  ],
  "attention_probs_dropout_prob": 0.1,
  "classifier_dropout": null,
  "gradient_checkpointing": false,
  "hidden_act": "gelu",
  "hidden_dropout_prob": 0.1,
  "hidden_size": 768,
  "id2label": {
    "0": "LABEL_0",
    "1": "LABEL_1",
    "2": "LABEL_2",
    "3": "LABEL_3",
    "4": "LABEL_4",
    "5": "LABEL_5",
    "6": "LABEL_6",
    "7": "LABEL_7"
  },
  "initializer_range": 0.02,
  "intermediate_size": 3072,
  "label2id": {
    "LABEL_0": 0,
    "LABEL_1": 1,
    "LABEL_2": 2,
    "LABEL_3": 3,
    "LABEL_4": 4,
    "LABEL_5": 5,
    "LABEL_6": 6,
    "LABEL_7": 7
  },
  "layer_norm_eps": 1e-12,
  "max_position_embeddings": 512,
  "model_type": "bert",
  "num_attention_heads": 12,
  "num_hidden_layers": 12,
  "pad_token_id": 0,
  "position_embedding_type": "absolute",
  "transformers_version": "4.15.0",
  "type_vocab_size": 2,
  "use_cache": true,
  "vocab_size": 30522
}

In [ ]:

from torch.utils.data import TensorDataset

#train set
dataset_train = TensorDataset(input_ids_train, 
                              attention_masks_train,
                              labels_train)

#validation set
dataset_val = TensorDataset(input_ids_val, 
                             attention_masks_val, 
                             labels_val)

In [ ]:
dataset_train

In [ ]:
print(len(dataset_train))
print(len(dataset_val))

9000
1000


In [ ]:
dataset_train.tensors

(tensor([[ 101, 3110, 1026,  ...,    0,    0,    0],
         [ 101, 1026, 1048,  ...,    0,    0,    0],
         [ 101, 1996, 2088,  ...,    0,    0,    0],
         ...,
         [ 101, 2720, 9353,  ...,    0,    0,    0],
         [ 101, 2074, 3427,  ...,    0,    0,    0],
         [ 101, 3492, 2469,  ...,    0,    0,    0]]),
 tensor([[1, 1, 1,  ..., 0, 0, 0],
         [1, 1, 1,  ..., 0, 0, 0],
         [1, 1, 1,  ..., 0, 0, 0],
         ...,
         [1, 1, 1,  ..., 0, 0, 0],
         [1, 1, 1,  ..., 0, 0, 0],
         [1, 1, 1,  ..., 0, 0, 0]]),
 tensor([1, 2, 1,  ..., 5, 4, 1]))

In [ ]:
from torch.utils.data import DataLoader, RandomSampler, SequentialSampler

batch_size = 32

#train set
dataloader_train = DataLoader(dataset_train,
                              sampler = RandomSampler(dataset_train),
                              batch_size = batch_size)

#validation set
dataloader_val = DataLoader(dataset_val,
                              sampler = RandomSampler(dataset_val),
                              batch_size = 32) #since we don't have to do backpropagation for this step


In [ ]:
from transformers import AdamW, get_linear_schedule_with_warmup

optimizer = AdamW(model.parameters(),
                 lr = 2e-5,
                 eps = 1e-8) #2e-5 > 5e-5
                 
epochs = 2

scheduler = get_linear_schedule_with_warmup(optimizer,
                                           num_warmup_steps = 0.1,
                                           num_training_steps = len(dataloader_train)*epochs)

In [ ]:
import numpy as np
from sklearn.metrics import f1_score

#f1 score
def f1_score_func(preds, labels):
    preds_flat = np.argmax(preds, axis=1).flatten()
    labels_flat = np.argmax(labels, axis=1).flatten()
    return f1_score(labels_flat, preds_flat, average = 'weighted')
#accuracy score
def accuracy_per_class(preds, labels):
    label_dict_inverse = {v: k for k, v in label_dict.items()}
    
    #make prediction
    preds_flat = np.argmax(preds, axis=1).flatten()
    labels_flat = labels.flatten()
    
    for label in np.unique(labels_flat):
        y_preds = preds_flat[labels_flat==label]
        y_true = labels_flat[labels_flat==label]
        print(f'Class: {label_dict_inverse[label]}')
        print(f'Accuracy:{len(y_preds[y_preds==label])}/{len(y_true)}\n')

In [ ]:
def evaluate(dataloader_val):

    #evaluation mode disables the dropout layer 
    model.eval()
    
    #tracking variables
    loss_val_total = 0
    predictions, true_vals = [], []
    
    for batch in tqdm(dataloader_val):
        
        #load into GPU
        batch = tuple(b.to(device) for b in batch)
        
        #define inputs
        inputs = {'input_ids':      batch[0],
                  'attention_mask': batch[1],
                  'labels':         batch[2]}

        #compute logits
        with torch.no_grad():        
            outputs = model(**inputs)
        
        #compute loss
        loss = outputs[0]
        logits = outputs[1]
        loss_val_total += loss.item()

        #compute accuracy
        logits = logits.detach().cpu().numpy()
        label_ids = inputs['labels'].cpu().numpy()
        predictions.append(logits)
        true_vals.append(label_ids)
    
    #compute average loss
    loss_val_avg = loss_val_total/len(dataloader_val) 
    
    predictions = np.concatenate(predictions, axis=0)
    true_vals = np.concatenate(true_vals, axis=0)
            
    return loss_val_avg, predictions, true_vals

In [ ]:
import numpy as np
from sklearn.metrics import f1_score

def f1_score_func(preds, labels):
    preds_flat = np.argmax(preds, axis = 1).flatten()
    labels_flat = labels.flatten()
    return f1_score(labels_flat, preds, average = 'weighted')

In [ ]:
#accuracy score
def accuracy_per_class(preds, labels):
    label_dict_inverse = {v: k for k, v in label_dict.items()}
    
    #make prediction
    preds_flat = np.argmax(preds, axis=1).flatten()
    labels_flat = labels.flatten()
    
    for label in np.unique(labels_flat):
        y_preds = preds_flat[labels_flat==label]
        y_true = labels_flat[labels_flat==label]
        print(f'Class: {label_dict_inverse[label]}')
        print(f'Accuracy:{len(y_preds[y_preds==label])}/{len(y_true)}\n')

In [ ]:
import random

seed_val = 123456
random.seed(seed_val)
np.random.seed(seed_val)
torch.manual_seed(seed_val)
torch.cuda.manual_seed_all(seed_val)

In [ ]:
# !pip install tqdm
from tqdm import tqdm

In [ ]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
model.to(device)
print(device)

cuda


In [ ]:
from tqdm import tqdm


for epoch in tqdm(range(4, epochs+1)):

    #set model in train mode
    model.train()

    #tracking variable
    loss_train_total = 0
    
    #set up progress bar
    progress_bar = tqdm(dataloader_train, 
                        desc='Epoch {:1d}'.format(epoch), 
                        leave=False, 
                        disable=False)
    
    for batch in progress_bar:
        #set gradient to 0
        model.zero_grad()

        #load into GPU
        batch = tuple(b.to(device) for b in batch)

        #define inputs
        inputs = {'input_ids': batch[0],
                  'attention_mask': batch[1],
                  'labels': batch[2]}
        
        outputs = model(**inputs)
        loss = outputs[0] #output.loss
        loss_train_total +=loss.item()

        #backward pass to get gradients
        loss.backward()
        
        #clip the norm of the gradients to 1.0 to prevent exploding gradients
        torch.nn.utils.clip_grad_norm_(model.parameters(), 1.0)
        
        #update optimizer
        optimizer.step()

        #update scheduler
        scheduler.step()
        
        progress_bar.set_postfix({'training_loss': '{:.3f}'.format(loss.item()/len(batch))})     
    
    tqdm.write('\nEpoch {epoch}')
    torch.save(model.state_dict(), f'/content/drive/MyDrive/BERT_ft_epoch{epoch}.model')
    #print training result
    loss_train_avg = loss_train_total/len(dataloader_train)
    tqdm.write(f'Training loss: {loss_train_avg}')
    
    #evaluate
    val_loss, predictions, true_vals = evaluate(dataloader_val)
    #f1 score
    # val_f1 = f1_score_func(predictions, true_vals)
    tqdm.write(f'Validation loss: {val_loss}')
    # tqdm.write(f'F1 Score (weighted): {val_f1}')

In [ ]:
# torch.save(model.state_dict(), f'/content/drive/MyDrive/BERT_ft_epoch{epoch}.model')
# torch.save(model.state_dict(), 'checkpoint.pth')

In [ ]:
# path = F"/content/gdrive/My Drive/{model_save_name}"

model.load_state_dict(torch.load('/content/drive/MyDrive/BERT_ft_epoch2.model'))

<All keys matched successfully>

In [ ]:
#save model
model.to(device)
pass

In [ ]:
#evaluate
_, predictions, true_vals = evaluate(dataloader_val)

100%|██████████| 32/32 [00:16<00:00,  1.89it/s]


In [ ]:
#get accuracy score
accuracy_per_class(predictions, true_vals)

Class: fear
Accuracy:12/48

Class: joy
Accuracy:247/352

Class: anticipation
Accuracy:97/175

Class: trust
Accuracy:62/134

Class: sadness
Accuracy:63/131

Class: disgust
Accuracy:48/94

Class: surprise
Accuracy:2/37

Class: anger
Accuracy:1/29



In [ ]:
test_data = pd.read_csv('/content/drive/MyDrive/testing_csv')
test_data.set_index("id",inplace=True)

In [ ]:
#tokenizer val set
encoded_data_test = tokenizer.batch_encode_plus(test_data.text.values,
                                                #add_special_tokens = True,
                                                return_attention_mask = True,
                                                pad_to_max_length = True,
                                                max_length = 150,
                                                return_tensors = 'pt')

In [ ]:
#encode val set
input_test =  encoded_test['input_ids']
attention_test = encoded_test['attention_mask']

In [ ]:
#encode val set
input_ids_test =  encoded_data_test['input_ids']
attention_masks_test = encoded_data_test['attention_mask']

In [ ]:
input_ids_test

tensor([[  101,  9657,  1997,  ...,     0,     0,     0],
        [  101,  1000,  3404,  ...,     0,     0,     0],
        [  101,  2043,  2079,  ...,     0,     0,     0],
        ...,
        [  101,  2043,  2017,  ...,     0,     0,     0],
        [  101,  1045,  8415,  ...,     0,     0,     0],
        [  101,  1030, 20463,  ...,     0,     0,     0]])

In [ ]:
len(attention_masks_test)

411972

In [ ]:
dataset_test = TensorDataset(input_ids_test, 
                             attention_masks_test
                             )

In [ ]:
dataset_test.tensors

(tensor([[  101,  9657,  1997,  ...,     0,     0,     0],
         [  101,  1000,  3404,  ...,     0,     0,     0],
         [  101,  2043,  2079,  ...,     0,     0,     0],
         ...,
         [  101,  2043,  2017,  ...,     0,     0,     0],
         [  101,  1045,  8415,  ...,     0,     0,     0],
         [  101,  1030, 20463,  ...,     0,     0,     0]]),
 tensor([[1, 1, 1,  ..., 0, 0, 0],
         [1, 1, 1,  ..., 0, 0, 0],
         [1, 1, 1,  ..., 0, 0, 0],
         ...,
         [1, 1, 1,  ..., 0, 0, 0],
         [1, 1, 1,  ..., 0, 0, 0],
         [1, 1, 1,  ..., 0, 0, 0]]))

In [ ]:
#load val set
dataloader_test = DataLoader(dataset_test) #since we don't have to do backpropagation for this step

In [ ]:
len(dataloader_test)

411972

In [ ]:
def pred(dataloader_test,label_dict):

    #evaluation mode disables the dropout layer 
    model.eval()
    
    #tracking variables
    loss_val_total = 0
    predictions = []
    iter = 32
    for batch in tqdm(dataloader_test):
        
        #load into GPU
        batch = tuple(b.to(device) for b in batch)
        
        #define inputs
        inputs = {'input_ids':      batch[0],
                  'attention_mask': batch[1]
                  }

        #compute logits
        with torch.no_grad():        
            outputs = model(**inputs)
        
        #compute loss
        # loss = outputs[0]
        logits = outputs[0]
        # loss_val_total += loss.item()

        #compute accuracy
        logits = logits.detach().cpu().numpy()

        #make prediction
        preds_flat = np.argmax(logits).flatten()

        predictions.append(list(label_dict.keys())[list(label_dict.values()).index(preds_flat)])
    
        if(len(predictions) == 10000):
          iter += 1
          ## read data from google drive
          DATASET_PATH = f'/content/drive/MyDrive/lab2_kaggle2/result{iter}_csv'

          ## get a sample to acelerate training
          df = pd.DataFrame(predictions)
          df.to_csv(DATASET_PATH, index=False, header=False)
          predictions = []
    if(len(predictions) > 0):
      iter += 1
      DATASET_PATH = f'/content/drive/MyDrive/lab2_kaggle2/result{iter}_csv'

      ## get a sample to acelerate training
      df = pd.DataFrame(predictions)
      df.to_csv(DATASET_PATH, index=False, header=False)
    
    # predictions = np.concatenate(predictions, axis=0)
    #make prediction

            
    return predictions

In [ ]:
print(len(dataloader_test))

411972


In [ ]:
label_dict

{'anger': 7,
 'anticipation': 2,
 'disgust': 5,
 'fear': 0,
 'joy': 1,
 'sadness': 4,
 'surprise': 6,
 'trust': 3}

In [ ]:
predictions = pred(dataloader_test,label_dict)

NameError: ignored

In [ ]:
DATASET_PATH = f'/content/drive/MyDrive/lab2_kaggle/result42_csv'

## get a sample to acelerate training
df = pd.DataFrame(predictions)
df.to_csv(DATASET_PATH, index=False, header=False)

In [ ]:
DATASET_PATH = f'/content/drive/MyDrive/lab2_kaggle/result{iter}_csv'

df_list = []
for iter in range(1, 43):
    df_list.append(pd.read_csv(f'/content/drive/MyDrive/lab2_kaggle/result{iter}_csv',header=None))

df = pd.concat(df_list)

In [ ]:
one = pd.read_csv(f'/content/drive/MyDrive/lab2_kaggle/result1_csv',header=None)

In [ ]:
len(df)

411972

In [ ]:
preds_flat = np.argmax(predictions[0]).flatten()
print(list(label_dict.keys())[list(label_dict.values()).index(preds_flat)])

joy


In [ ]:
## read data from google drive
DATASET_PATH = '/content/drive/MyDrive/result_csv'

## get a sample to acelerate training 
predictions = pd.to_csv(DATASET_PATH)

In [ ]:
def pred_to_label(preds):

    #make prediction
    preds_flat = np.argmax(preds, axis=1).flatten()

    return preds_flat

In [ ]:
pred_label = pred_to_label(preds_flat)

In [ ]:
test_data.

In [ ]:
## read data from google drive
DATASET_PATH = '/content/drive/MyDrive/result_csv'

## get a sample to acelerate training 
predictions = pd.to_csv(DATASET_PATH)

  0%|          | 0/12875 [00:00<?, ?it/s]


IndexError: ignored